In [ ]:
#Import libraries

import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
#Base training directory path
base_dir = "[INSERT PATH]"

#Path to validation folder (unblurred images)
validation_dir = "[INSERT PATH]"

#Intialize counter
count = 0

# List of directories for training with different blur levels
train_dirs = [
    base_dir + "[3% PATH]"
    base_dir + "[10% PATH]"
    base_dir + "[25% PATH]"
]

In [ ]:
#Load data in batches
def load_data(path, image_size=(64, 64), batch_size=300):
    generator = ImageDataGenerator().flow_from_directory(
        directory=path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    return generator

In [ ]:
#Define ResNet50 model
def build_model(image_size, num_classes):
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
    model = Sequential([
        resnet,
        Flatten(),
        Dense(256, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    for layer in resnet.layers:
        layer.trainable = False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#Train CNN and log results
def train_model(model, train_generator, validation_generator, epochs=10):
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs
    )
    return history


In [ ]:
#Output found classes
validation_generator = load_data(validation_dir)

In [ ]:
#Loop through each training directory and train a model for each
for train_dir in train_dirs:
    count += 1
    print(f"Processing directory: {train_dir}")
    train_generator = load_data(train_dir)
    num_classes = len(train_generator.class_indices)
    model = build_model(image_size=(64, 64), num_classes=num_classes)
    history = train_model(model, train_generator, validation_generator)
    
    #Print accuracy for each model
    accuracy = history.history['val_accuracy'][-1]
    print(f"Validation Accuracy for {train_dir}: {accuracy:.4f}")

    #Append accuracy to a file
    with open(f'results{count}.txt', 'a') as file:
        file.write(f"Validation Accuracy for {train_dir}: {accuracy:.4f}\n")

In [ ]:
#Function to output training history plots
def plot_history(history):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(history.history['accuracy'], label='Train Accuracy')
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0].set_title('Model Accuracy')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()

    axes[1].plot(history.history['loss'], label='Train Loss')
    axes[1].plot(history.history['val_loss'], label='Validation Loss')
    axes[1].set_title('Model Loss')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Loss')
    axes[1].legend()

    plt.tight_layout()
    plt.show()
    plt.savefig(f'output{count}.png')

#Plot the training history
plot_history(history)
